In [1]:
import tensorflow as tf

$$y = Ax^2 + Bx + c$$

In [3]:
class Quadratic(tf.keras.layers.Layer):
    
    def __init__(self, units, activation=None):
        super(Quadratic, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        a_init = tf.random_normal_initializer()
        self.A = tf.Variable(
            name='kernel_A',
            initial_value=a_init(
                shape=(input_shape[-1], self.units), dtype=tf.float32
            ),
            trainable=True
        )
        b_init = tf.random_normal_initializer()
        self.B = tf.Variable(
            name='kernel_B',
            initial_value=b_init(
                shape=(input_shape[-1], self.units), dtype=tf.float32
            ),
            trainable=True
        )
        c_init = tf.zeros_initializer()
        self.C = tf.Variable(
            name='kernel_C',
            initial_value=c_init(
                shape=(self.units, ), dtype=tf.float32
            ),
            trainable=True
        )
        
        super().build(input_shape)
    
    def call(self, inputs):
        return self.activation(
            tf.matmul(tf.square(inputs), self.A) + tf.matmul(inputs, self.B) + self.C
        )

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_custom_model():
    inp = tf.keras.Input((28, 28))
    x = tf.keras.layers.Flatten()(inp)
    x = Quadratic(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = Quadratic(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = Quadratic(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = Quadratic(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inp, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
model = create_custom_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
quadratic (Quadratic)        (None, 256)               401664    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
quadratic_1 (Quadratic)      (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
quadratic_2 (Quadratic)      (None, 64)                16448 

In [7]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6224 - accuracy: 0.8547
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2978 - accuracy: 0.9271
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2942 - accuracy: 0.9320
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3125 - accuracy: 0.9345
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.1701 - accuracy: 0.9553


[0.17011846601963043, 0.955299973487854]